In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-258368
Azure region: eastus2
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-258368


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-4-standard-d2-v2"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # from https://github.com/Azure/MachineLearningNotebooks/blob/master//how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.ipynb
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=0, max_nodes=4, idle_seconds_before_scaledown=300)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# examples used in this code:

# from https://learn.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters?view=azureml-api-1
# and https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/keras/train-hyperparameter-tune-deploy-with-keras/train-hyperparameter-tune-deploy-with-keras.ipynb

# Specify parameter sampler

#from train.py, the parameters accepted are --C and --max_iter
# meaning of parameters, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# the "C" hyperparameter controls the strength of the regularization. A smaller value for "C" (e.g. C=0.01) leads to stronger regularization and a simpler model, while a larger value (e.g. C=1.0) leads to weaker regularization and a more complex model.
# max_iter, default=100 - Maximum number of iterations taken for the solvers to converge. 

ps = RandomParameterSampling( {
        "--C": uniform(0.01, 1.0),
        "--max_iter": choice(50, 100, 200, 300)
    })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src =ScriptRunConfig(source_directory=".",
                            script="train.py",
                            # arguments=['--C', 1.0, '--max_iter', 100],  default parameter values, tunned by hyperdrive
                            compute_target=cluster,
                            environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# we will stop in 100 runs or 30 minutes, what happens first
# here is important to write the primary metric name as "Accuracy", and not "accuracy" as it was in the MS documentation, because our train.py script use with upper case, 
# and the call  hyperdrive_run.get_best_run_by_primary_metric() return None if the case is incorrect.

hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=100,
                             max_duration_minutes=30)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment_name="Bankmarketing-Hyperdrive"
experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_config)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
# ensure that the job has finished

hyperdrive_run.wait_for_completion(show_output=True)

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[1])
print('\n max_iter:',parameter_values[3])
print('\n filenames:',best_run.get_file_names())

Best Run Id:  HD_7f1d16be-644c-439c-b269-41b4f820a8d8_29

 Accuracy: 0.9114820435002529

 C: 0.7453876768284908

 max_iter: 100

 filenames: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2024-04-26-17', 'outputs/model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [9]:
# see the best run in web interface

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Bankmarketing-Hyperdrive,HD_7f1d16be-644c-439c-b269-41b4f820a8d8_29,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
# save on AzureML

model = best_run.register_model(model_name='Bankmarketing-Hyperdrive-bestmodel', model_path='outputs/model.pkl')

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [15]:
#  Restart kernel if not find train.py - https://stackoverflow.com/questions/76424178/import-python-module-in-azure-machine-learning

from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [16]:
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

# from https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train?view=azureml-api-1&tabs=python
# training_data cannot be the source dataset, because it will ignore the cleandata, but cleandata separates x (attributes) and y (prediction)
# so, we concatenate the series y in the dataframe x again to send to automl_config

x['y'] = y

# but it is not possible to pass a dataframe as training data for a automl, it generates the error:
# Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]
# so, save the dataframe as CSV, upload the file to the datastore and create a tabular dataset with the cleaning data:

x.to_csv("bankmarketing_train_cleaned.csv",index=False)

# upload the file do blob store - https://stackoverflow.com/questions/76278179/how-can-we-upload-local-files-to-azure-ml-studio-notebooks-using-python

bds = ws.get_default_datastore()
bds.upload_files(files=["bankmarketing_train_cleaned.csv"], target_path='\\', overwrite=True)

# how to use paths from datastore - https://stackoverflow.com/questions/69582785/create-tabular-dataset-in-azure-using-python-sdk

ds_clean = TabularDatasetFactory.from_delimited_files(path=[ (bds,"bankmarketing_train_cleaned.csv") ] )
ds_clean

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading bankmarketing_train_cleaned.csv
Uploaded bankmarketing_train_cleaned.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', 'bankmarketing_train_cleaned.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [24]:
from azureml.train.automl import AutoMLConfig

# it is possible to pass training data and validation data separated in AutoML configuration (using train_test_split from SkLearn), but we prefer to use cross validation.
# details about of K-fold cross-validation: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits?view=azureml-api-1
# I use the value 5, meaning 5 experiments with 80% training data and 20% test data, alternating between data segments.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = cluster,
    task="classification",
    primary_metric="accuracy",
    training_data=ds_clean,
    label_column_name="y",
    n_cross_validations=5)

In [25]:
# Submit the automl run

experiment_name="Bankmarketing-AutoML"

experiment = Experiment(ws, experiment_name)
automl_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cluster-4-standard-d2-v2 with default configuration
Running on remote compute: cluster-4-standard-d2-v2


Experiment,Id,Type,Status,Details Page,Docs Page
Bankmarketing-AutoML,AutoML_d9c1b784-8d8b-45e1-94be-0ca71054506b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [33]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Bankmarketing-AutoML,AutoML_d9c1b784-8d8b-45e1-94be-0ca71054506b,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [34]:
# see the running job details

from azureml.widgets import RunDetails
from azureml.core.run import Run
experiment = Experiment(ws, experiment_name)
run_id = "AutoML_d9c1b784-8d8b-45e1-94be-0ca71054506b"
autorun = Run(experiment, run_id)
RunDetails(autorun).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# ensure that the job has finished

automl_run.wait_for_completion(show_output=True)

In [27]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(fitted_model.steps)

# best model metrics

automl_run_metrics = automl_run.get_metrics()
print('Accuracy', automl_run_metrics['accuracy'])

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

[('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('21', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_

In [31]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Bankmarketing-AutoML,AutoML_d9c1b784-8d8b-45e1-94be-0ca71054506b_31,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [30]:
# save on AzureML

automl_run.register_model(model_name="Bankmarketing-AutoML-bestmodel")

# save to disk

import joblib
joblib.dump(fitted_model, 'Bankmarketing-AutoML-bestmodel.pkl')

['Bankmarketing-AutoML-bestmodel.pkl']

In [36]:
# Deletion of compute cluster

cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

